# Домашнее задание 3. Торговый робот

### Панкратов Никита

### Импортируем нужные библиотеки

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import yfinance as yf
import datetime as dt

import plotly
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
from plotly.subplots import make_subplots
init_notebook_mode(connected=True)

## Исходные данные

In [2]:
money = 1000000 # наш исходный банк
start = dt.datetime(2015,1,1)   # начальная дата наблюдения
end = dt.datetime(2020, 12, 31) # конечная дата наблюдения

stop_loss = 0.05 # стоп-лосс

short_window = 30 # размер короткой MA=30
long_window = 90  # размер длинной MA=90


## Скачиваем (вставляем необходимую базу котирвоок для построения дальнейшей стратегии) 


# P.s. просьба, после предъявления фрейма присвоить его переменной df_stock!

In [3]:
#df_stock = yf.download(tickers='BNB-USD', start=start, end=end, period='1d').iloc[:, :-2]
#df_stock = 

[*********************100%***********************]  1 of 1 completed


# Основная функция (for Кирилл Юрьевич: график можно увеличить в ширине при выборке конкретного куска: НЕ через движущийся ползунок, а через верхний график, путем выделяния левой кнопкой мыши нужного окна)

In [4]:
def MA_trade(df_stock, dep, stop_loss, short_window, long_window):
    short_window_values = df_stock.rolling(short_window).mean().copy()  # 30 days
    long_window_values = df_stock.rolling(long_window).mean().copy()    # 90 days
    df_review = pd.DataFrame(columns=['date', 'signal', 'num_shares', "share_price", "share_value", "cash"])
    num_shares = 0
    cash = dep
    share_value = 0
    sig_buy = 0
    sig_sell = 0
    open_pos = 'NO'
    
    signal_x_values = []
    signal_y_values = []
    signal_colors = []
    
    for i in range(len(df_stock)):  
        if open_pos == 'NO':
            if (i!=0) and (sig_buy == 0) and (short_window_values.iloc[i,3] > long_window_values.iloc[i,3]) and (short_window_values.iloc[i-1,3] < long_window_values.iloc[i-1,3]):
                sig_buy = 1
                df_review.loc[len(df_review)] = [df_stock.index[i], 'sig_buy', num_shares, df_stock.iloc[i,3], 0, cash] # заявка на покупку 
            elif sig_buy == 1:
                num_shares = cash // df_stock.iloc[i, 0]  # делим нацело (1-open, берем на момент открытия торгов следующего дня)
                share_value = num_shares * df_stock.iloc[i,0] 
                cash = round(cash - share_value, 2) # банковское окргуление до двух знаков после запятой
                df_review.loc[len(df_review)] = [df_stock.index[i], 'buy', num_shares, df_stock.iloc[i,0], round(share_value, 2), cash]  # покупаем (открытие ЛОНГ-позиции)
                stop_price = (1-stop_loss) * df_stock.iloc[i,0]
                sig_buy = 0
                open_pos = 'YES'
                
                #for visual.
                signal_x_values.append(df_stock.index[i])  # дата
                signal_y_values.append(df_stock.iloc[i,0]) # значение (котировка акции)
                signal_colors.append('green')
                
        if open_pos == 'YES': # так как внутри дня, уже может срабоать стоп-лосс/сигнал для продажи
            if (sig_sell != 1) and (df_stock.iloc[i,0] < stop_price or df_stock.iloc[i,2] < stop_price): # условие отработки стопл-лосса (low текущего дня)
                if df_stock.iloc[i,0] < stop_price:  # если цена открытия ментше low
                    share_value = num_shares * df_stock.iloc[i,0]  # закрываю в этот день по цене открытия
                    cash = round(cash+share_value, 2)                          
                    df_review.loc[len(df_review)] = [df_stock.index[i], 'stop-loss', num_shares, df_stock.iloc[i,0], 0, cash] # здесь индекс через len, так как i - может быть одинаковым
                    num_shares = 0 
                    open_pos = 'NO'
                    share_value = 0 
                    
                    #for visual
                    signal_x_values.append(df_stock.index[i+1])  
                    signal_y_values.append(df_stock.iloc[i+1,0])
                    signal_colors.append('black')
                    
                else: # если low был ниже stop_loss или close (все равно), заерываю в этот же день по цене stop_loss-а
                    share_value = num_shares * stop_price
                    cash = round(cash+share_value, 2)                          
                    df_review.loc[len(df_review)] = [df_stock.index[i], 'stop-loss', num_shares, stop_price, 0, cash]
                    num_shares = 0 
                    open_pos = 'NO'
                    share_value = 0 
                    
                    #for visual
                    signal_x_values.append(df_stock.index[i])  
                    signal_y_values.append(stop_price) 
                    signal_colors.append('black')
                    
                    
            elif (sig_sell == 0) and (short_window_values.iloc[i,3] < long_window_values.iloc[i,3]):
                share_value = num_shares * df_stock.iloc[i,3]
                df_review.loc[len(df_review)] = [df_stock.index[i], 'sig-sale', num_shares, df_stock.iloc[i,3], round(share_value, 2), cash]
                sig_sell = 1
            elif sig_sell == 1:
                share_value = num_shares * df_stock.iloc[i,0]
                cash = round(cash + share_value, 2)
                df_review.loc[len(df_review)] = [df_stock.index[i], 'sale', num_shares, df_stock.iloc[i,0], 0, cash]
                num_shares = 0
                sig_sell = 0
                open_pos = 'NO'
                share_value = 0 
                
                #for visual
                signal_x_values.append(df_stock.index[i])  
                signal_y_values.append(df_stock.iloc[i,0])
                signal_colors.append('red')
    
    trace = go.Ohlc(x=df_stock.index,
                open=df_stock['Open'],
                high=df_stock['High'],
                low=df_stock['Low'],
                close=df_stock['Close'],
                   name='Ohlc свечи')
    
    # Создаем MA(30)
    ma30_trace = go.Scatter(x=df_stock.index,
                            y=df_stock['Close'].rolling(window=30).mean(),
                            mode='lines',
                            name='MA(30)')

    # Создаем MA(90)
    ma60_trace = go.Scatter(x=df_stock.index,
                            y=df_stock['Close'].rolling(window=90).mean(),
                            mode='lines',
                            name='MA(90)')
    
    symbols_treang = []
    for i in signal_colors:
        if i == 'green':
            symbols_treang.append('triangle-up')
        else:
            symbols_treang.append('triangle-down')
    
    # Сигналы покупки/продажи (зел., красн., черные треугольнички)
    signal_trace = go.Scatter(x=signal_x_values,
                          y=signal_y_values,  
                          mode='markers',
                          marker=dict(symbol=symbols_treang,  
                                      color=signal_colors, 
                                      size=12),
                          name='Сигналы')
    
    # Делаем два окна визуализации
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True)
    
    #Размеры графиков
    fig.update_layout(
        height=800,   
        width=1000,   
    )
    
    # 1-ое окно (price, MA, signals)
    fig.add_trace(trace, row=1, col=1)
    fig.add_trace(ma30_trace, row=1, col=1)
    fig.add_trace(ma60_trace, row=1, col=1)
    fig.add_trace(signal_trace, row=1, col=1)
    
    #Обновляем название первого графика
    fig.update_yaxes(title_text=f'Price of some_share', row=1, col=1)
    
    # 2-ое окно (in trade or not)
    # Создаем трейсы для второго окна с линиями
    buy_signals = [signal_x_values[i] for i in range(len(signal_x_values)) if signal_colors[i] == 'green']
    sell_signals = [signal_x_values[i] for i in range(len(signal_x_values)) if signal_colors[i] in ['red', 'black']]

    fig.add_trace(go.Scatter(x=buy_signals, y=[1] * len(buy_signals),
                             mode='text',
                             line=dict(color='green', width=1),
                             name='Сигналы на покупку'), row=2, col=1)

    fig.add_trace(go.Scatter(x=sell_signals, y=[0] * len(sell_signals),
                             mode='text',
                             line=dict(color='red', width=1),
                             name='Сигналы на закрытие'), row=2, col=1)
    
    # Настраиваем второе окно
    fig.update_yaxes(title_text='Сигналы', row=2, col=1, range=[-0.1, 2.1], tickvals=[0,1])
    fig.update_xaxes(title_text='Дата', row=2, col=1)

    # Добавляем вертикальные линии для сигналов на покупку
    for buy_signal in buy_signals:
        fig.add_shape(type='line',
                      x0=buy_signal, y0=1, x1=buy_signal, y1=2,
                      line=dict(color='green', width=1, dash='dash'),
                      row=2, col=1)
        
        fig.add_shape(type='line',
                      x0=buy_signal, y0=0, x1=buy_signal, y1=1,
                      line=dict(color='black', width=1.2),
                      row=2, col=1)

    # Добавляем вертикальные линии для сигналов на закрытие
    for sell_signal in sell_signals:
        fig.add_shape(type='line',
                      x0=sell_signal, y0=2, x1=sell_signal, y1=1,
                      line=dict(color='red', width=1, dash='dash'),
                      row=2, col=1)
        
        fig.add_shape(type='line',
                      x0=sell_signal, y0=1, x1=sell_signal, y1=0,
                      line=dict(color='black', width=1.2),
                      row=2, col=1)
    
    # Добавляем горизонтальные линии на уровне y=1 (если мы в сделке) и y=0, в противном случае
    fig.add_shape(type='line',
                          x0=df_stock.index[0], y0=0, x1=buy_signals[0], y1=0,
                          line=dict(color='black', width=1.2),
                          row=2, col=1)
    
    for i in range(len(buy_signals)):
        try:
            buy_signal = buy_signals[i]
            sell_signal = sell_signals[i]

            fig.add_shape(type='line',
                          x0=buy_signal, y0=1, x1=sell_signal, y1=1,
                          line=dict(color='black', width=1.2),
                          row=2, col=1)
            try: # если сделка остается открытой к концу периода
                fig.add_shape(type='line',
                          x0=sell_signals[i], y0=0, x1=buy_signals[i+1], y1=0,
                          line=dict(color='black', width=1.2),
                          row=2, col=1)
            except: # если сделки нет до конца
                fig.add_shape(type='line',
                          x0=sell_signals[i], y0=0, x1=df_stock.index[-1], y1=0,
                          line=dict(color='black', width=1.2),
                          row=2, col=1)
        except:
            fig.add_shape(type='line',
                          x0=buy_signal, y0=1, x1=df_stock.index[-1], y1=1,
                          line=dict(color='black', width=1.2),
                          row=2, col=1)
   
    
    # Для обновления зума
    fig.update_layout(
            xaxis=dict(
            rangeslider=dict(visible=True),
            type="date" 
        ),
        yaxis=dict(fixedrange=False), 
        dragmode="zoom", 
    )
    
    
    # Отображаем график
    iplot(fig)
    
    print(f"Процент прироста составил {((cash+share_value)-dep)/dep * 100}%!")
    
    if open_pos == 'YES':
        if df_review.iloc[-1,1] == 'buy': # последняя дествие - это покупка
            return df_review.iloc[0:-2,:]
        elif df_review.iloc[-1,1] == 'sig_buy': # последняя дествие - это СИГНАЛ на покупку (могло быть, например, в самом последнем дне, поэтому считай 'buy' могло даже и не быть)
            return df_review.iloc[0:-1,:]
        elif df_review.iloc[-1,1] == 'sig_sell': # последняя дествие - это СИГНАЛ на продажу (могло быть, например, в самом последнем дне, поэтому считай 'sell' просто не успели сделать (нет данных на след день))
            return df_review.iloc[0:-3,:]
    else:
        return df_review


In [5]:
result = MA_trade(df_stock, dep=money, stop_loss=stop_loss, short_window=short_window, long_window=long_window)
result

Процент прироста составил 235.0610812621155%!


,date,signal,num_shares,share_price,share_value,cash
0,2018-04-12,sig_buy,0.0,12.952500,0.00,1000000.00
1,2018-04-13,buy,76031.0,13.152500,999997.74,2.26
2,2018-04-16,stop-loss,76031.0,12.494875,0.00,950000.11
3,2019-02-01,sig_buy,0.0,6.562617,0.00,950000.11
4,2019-02-02,buy,144942.0,6.554346,950000.03,0.08
5,2019-07-30,sig-sale,144942.0,26.942305,3905071.51,0.08
6,2019-07-31,sale,144942.0,26.962410,0.00,3907985.71
7,2019-11-21,sig_buy,0.0,16.840218,0.00,3907985.71
8,2019-11-22,buy,232053.0,16.840914,3907984.56,1.15
9,2019-11-22,stop-loss,232053.0,15.998868,0.00,3712586.49
